## InBody Graph

#### WT: 체중 kg
#### SMM: 골격근량 kg
#### BFM: 체지방  kg
#### PFM: 체지방률 %

In [ ]:
import ssl
from urllib.request import Request, urlopen
from datetime import datetime

def crawling(url='', encoding='utf-8', err=lambda e: print(f'{e} : {datetime.now()}', file=sys.stderr)):
    try:
        req = Request(url)
        ssl._create_default_https_context = ssl._create_unverified_context

        resp = urlopen(req)
        recv = resp.read().decode(encoding, errors='replace')

        print(f'{datetime.now()}: success for request [{url}]')
        return recv
    except Exception as e:
        err(e)

In [ ]:
import sys
from functools import reduce
from utils import crawling

resfetch = list(filter(
    lambda s: s.startswith('#'),
    reduce(
        lambda param, func: func(param),
        [lambda txt: [] if txt is None else txt.split('\n'), lambda strs: [f(s) for f in [str.strip] for s in strs]],
        crawling('https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/InBody.md')
    )
))

if len(resfetch) == 0:
    sys.exit(0)

print(resfetch)

In [ ]:
import re

resarrange = []

for res in resfetch:
    if res.startswith('####'):
        item, val = (s.strip() for s in res.replace('####', '').strip().split(':'))
        sval, unit = re.compile(r'(\d+.\d*)\s*(.*)').match(re.sub(r"[\([{})\]\s]", "", val)).groups()
        resarrange.append((date, item, float(sval), unit))
    elif res.startswith('##'):
        date = res.replace('##', '').strip()

print(resarrange)

In [ ]:
from datetime import datetime

dataset_x, dataset_y, d, key = [], {}, '', ''
for result in resarrange:
    if d != result[0]:
        d = result[0]
        dataset_x.append(datetime.strptime(d, '%Y.%m.%d'))

    if key != result[1]:
        key = result[1]
        if key not in dataset_y:
            dataset_y.update({key: {'unit': result[3], 'values': []}})
        dataset_y[key]['values'].append(result[2])

print(dataset_x)
print(dataset_y)

In [ ]:
from itertools import cycle
from matplotlib import pyplot as plt

dataset_x_days = [(x - dataset_x[0]).days for x in dataset_x]
dataset_x_fdates = [x.strftime('%m.%d') for x in dataset_x]

legendhandles = []
cycol = cycle('bgrcmk')

fig = plt.figure(figsize=(30, 10))
spec = plt.GridSpec(1, 1)
subplot = fig.add_subplot(spec[0, 0])

subplot.set_yticks([])
subplot.set_xticks(dataset_x_days)
subplot.set_xticklabels(dataset_x_fdates, fontsize=9)

for key in dataset_y.keys():
    tx = subplot.twinx()

    tx.set_yticks([])
    line, = tx.plot(dataset_x_days, dataset_y[key]['values'], 'o-', color=next(cycol), label=f"{key}({dataset_y[key]['unit']})")

    for idx, yv in enumerate(dataset_y[key]['values']):
        tx.annotate(str(yv), xy=(dataset_x_days[idx], yv), xytext=(-10, 5), textcoords='offset points')

    legendhandles.append(line)

plt.legend(handles=legendhandles, loc='lower center')
plt.show()